In [1]:
from catboost import datasets

import pandas as pd

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

import numpy as np

import warnings
warnings.simplefilter('ignore')

#### Загрузка датасетов задачи "Titanic disaster" (https://www.kaggle.com/c/titanic), содержащегося в библиотеке категориального бустинга от Яндекса catboost.

In [2]:
df, dft = datasets.titanic()

In [3]:
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


#### Устранение пустых (NaN) значений

In [4]:
display(pd.DataFrame(df.isna().sum()).T)
display(pd.DataFrame(dft.isna().sum()).T)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,0,0,0,0,177,0,0,0,0,687,2


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,0,0,0,86,0,0,0,1,327,0


In [5]:
df.Age.fillna(df.Age.mean(), inplace=True)
dft.Age.fillna(dft.Age.mean(), inplace=True)
df.Embarked.fillna('Q', inplace=True)
dft.Fare.fillna(dft.Fare.mean(), inplace=True)

#### Формирование матрицы признаков, создание новых признаков методом one-hot-encoding для категориальных переменных

In [6]:
df = pd.get_dummies(df[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']])
dft = pd.get_dummies(dft[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']])

In [7]:
X = df.drop(['Survived'], axis=1)
y = df.Survived
X_predict = dft

#### Создание и обучение модели логистической регрессии для предсказания с использованием отложенной выборки (train - данные для обучения, test - данные для тестирования качества работы обученной модели)

In [8]:
model = LogisticRegression()

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=True)

In [10]:
model.fit(X_train, y_train)

LogisticRegression()

In [11]:
model.score(X_test, y_test)

0.7661016949152543

#### Подбор гиперпараметров для модели логистической регрессии с помощью функции GridSearchCV

In [12]:
params = {'C': np.linspace(0, 100, 100)}

In [13]:
model_grid = GridSearchCV(model, param_grid=params, cv=10, n_jobs=-1, verbose=1)

In [14]:
model_grid.fit(X, y)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 962 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    5.1s finished


GridSearchCV(cv=10, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([  0.        ,   1.01010101,   2.02020202,   3.03030303,
         4.04040404,   5.05050505,   6.06060606,   7.07070707,
         8.08080808,   9.09090909,  10.1010101 ,  11.11111111,
        12.12121212,  13.13131313,  14.14141414,  15.15151515,
        16.16161616,  17.17171717,  18.18181818,  19.19191919,
        20.2020202 ,  21.21212121,  22.22222222,  23.23232323...
        68.68686869,  69.6969697 ,  70.70707071,  71.71717172,
        72.72727273,  73.73737374,  74.74747475,  75.75757576,
        76.76767677,  77.77777778,  78.78787879,  79.7979798 ,
        80.80808081,  81.81818182,  82.82828283,  83.83838384,
        84.84848485,  85.85858586,  86.86868687,  87.87878788,
        88.88888889,  89.8989899 ,  90.90909091,  91.91919192,
        92.92929293,  93.93939394,  94.94949495,  95.95959596,
        96.96969697,  97.97979798,  98.98989899, 100.        ])},
             verbose

In [15]:
model_grid.best_estimator_, model_grid.best_score_, model_grid.best_params_

(LogisticRegression(C=79.7979797979798),
 0.8013607990012485,
 {'C': 79.7979797979798})

Получили более высокую точность, чем при использовании отложенной выборки, определив значение гиперпараметра $C=79.79$.